In [60]:
# initialize 
from datetime import datetime
import pandas as pd
from pybaseball import statcast, cache, playerid_lookup, statcast_pitcher, statcast_batter,statcast_batter_expected_stats,statcast_pitcher_expected_stats,batting_stats, pitching_stats, team_pitching, team_batting, team_fielding, standings, playerid_reverse_lookup, playerid_reverse_lookup
cache.enable()
start_dt = '2024-03-28'
end_dt = datetime.now().date().strftime('%Y-%m-%d')

In [61]:
batters = [
    "Logan O'Hoppe",
    "Justin Turner",
    "Ketel Marte",
    "Max Muncy",
    "Ezequiel Tovar",
    "Juan Soto",
    "Steven Kwan",
    "Michael Conforto",
    "Aaron Judge",
    "Tyler O'Neill",
    # "Luis Robert Jr" # gotta figure out the suffix
]
pitchers = [
    "Yoshinobu Yamamoto",
    "Josh Hader",
    "Sonny Gray",
    "Kenley Jansen",
    "Cristian Javier",
    "Hunter Greene",
    "Carlos Rodon",
    "Aaron Nola",
    "Ronel Blanco",
    "Nick Pivetta",
]

batter_ids = pd.concat([
    playerid_lookup(split_name[1], split_name[0])
    for split_name in [name.split(" ") for name in batters]
])
pitcher_ids = pd.concat([
    playerid_lookup(split_name[1], split_name[0])
    for split_name in [name.split(" ") for name in pitchers]
])
roster_ids = pd.concat([batter_ids, pitcher_ids])

In [75]:
def getRosterStats(player_ids:pd.DataFrame, statcast_func, start_dt=start_dt, end_dt=end_dt):
    key_mlbams = player_ids[['key_mlbam']].values.tolist()
    player_stats = [statcast_func(start_dt, end_dt, player_id=key_mlbam[0]) for key_mlbam in key_mlbams if key_mlbam[0] is not None]
    df = pd.concat(player_stats)
    df['game_date'] = pd.to_datetime(df['game_date'])
    return df

In [76]:
batter_stats = getRosterStats(roster_ids, statcast_batter)
pitcher_stats = getRosterStats(roster_ids, statcast_pitcher)

[[681351], [457759], [606466], [571970], [678662], [665742], [680757], [624424], [592450], [641933], [808967], [623352], [543243], [445276], [664299], [668881], [605400], [669854], [601713]]
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


C:\Users\kylebrague\AppData\Local\Temp\ipykernel_46492\542853754.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(player_stats)


[[681351], [457759], [606466], [571970], [678662], [665742], [680757], [624424], [592450], [641933], [808967], [623352], [543243], [445276], [664299], [668881], [605400], [669854], [601713]]
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


C:\Users\kylebrague\AppData\Local\Temp\ipykernel_46492\542853754.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(player_stats)


In [77]:

average_velocity = pitcher_stats[['player_name','release_speed','pitch_name']] \
  .groupby(['player_name','pitch_name']) \
  .agg(['mean']) \
  .sort_values(by=('release_speed','mean'),ascending=False)

average_velocity.style.set_caption("Average Pitch Velocity by Pitcher").format(precision=3)

In [78]:
average_velocity.to_html('exports/average_velocity.html')

In [79]:
batter_stats.to_csv('exports/batter_stats.csv',index=False)
pitcher_stats.to_csv('exports/pitcher_stats.csv',index=False)

In [91]:
fg_batter_ids = batter_ids['key_fangraphs'].values.tolist()
fg_pitcher_ids = pitcher_ids['key_fangraphs'].values.tolist()
print(fg_batter_ids)
print(fg_pitcher_ids)
batting_stats_fg = batting_stats(2024).set_index('IDfg').filter(items=fg_batter_ids, axis=0)
pitching_stats_fg = pitching_stats(2024).set_index('IDfg').filter(items=fg_pitcher_ids, axis=0)

[24729, 5235, 13613, 13301, 24064, 20123, 24610, 16376, 15640, 15711]
[-1, 14212, 12768, 3096, 17606, 22182, 16149, 19407, 15454]


,Season,Name,Team,Age,G,AB,PA,H,1B,2B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
24729,2024,Logan O'Hoppe,LAA,24,14,44,51,16,12,2,...,111.2,21,0.600,35,0.165,0.280,0.342,0.573,0.434,0.8
5235,2024,Justin Turner,TOR,39,15,47,58,17,9,7,...,106.8,18,0.462,39,0.165,0.243,0.338,0.522,0.432,0.7
13613,2024,Ketel Marte,ARI,30,17,67,77,18,10,5,...,114.4,33,0.569,58,0.124,0.217,0.310,0.521,0.391,0.4
13301,2024,Max Muncy,LAD,33,18,65,77,17,9,4,...,110.0,19,0.475,40,0.181,0.328,0.230,0.453,0.335,0.6
24064,2024,Ezequiel Tovar,COL,22,17,64,71,19,12,4,...,106.5,21,0.477,44,0.117,0.307,0.234,0.424,0.314,0.4
20123,2024,Juan Soto,NYY,25,17,64,81,21,15,3,...,115.7,31,0.564,55,0.188,0.252,0.309,0.538,0.433,1.1
24610,2024,Steven Kwan,CLE,26,16,73,75,26,22,2,...,103.2,10,0.161,62,0.233,0.280,0.341,0.453,0.354,0.6
16376,2024,Michael Conforto,SFG,31,16,61,66,18,9,5,...,109.9,23,0.489,47,0.182,0.284,0.281,0.567,0.385,0.6
15640,2024,Aaron Judge,NYY,32,17,62,79,12,4,5,...,114.6,23,0.500,46,0.158,0.261,0.251,0.489,0.392,0.4
15711,2024,Tyler O'Neill,BOS,29,15,48,61,15,8,0,...,110.6,16,0.457,35,0.201,0.274,0.270,0.630,0.435,0.9


In [100]:
batter_dashboard = batting_stats_fg[['Name','PA','HR','R','RBI','SB','BB%','K%','ISO','BABIP','AVG','OBP','SLG','OPS','wOBA','wRC+','BsR','WAR','Off','Def',]]
batter_dashboard.head(10).sort_values(by='OPS',ascending=False).to_html('exports/batter_dashboard.html')

In [92]:
pitching_stats_fg.to_csv('exports/pitching_stats_fg.csv',index=False)
batting_stats_fg.to_csv('exports/batting_stats_fg.csv',index=False)

In [68]:
start_dt = '2024-03-28'
end_dt = datetime.now().date().strftime('%Y-%m-%d')
statcast(start_dt, end_dt, ).to_csv(f'exports/statcast_{start_dt}-{end_dt}.csv', index=False)

This is a large query, it may take a moment to complete


100%|██████████| 20/20 [00:00<00:00, 28.04it/s]
c:\Users\kylebrague\Code\Sandboxes\baseball\.venv\Lib\site-packages\pybaseball\statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)
